# Harmonic Signals
### Observe waveform and spectrum or spectrogram 
### Control frequency and amplitude of a sine - square - sawtooth signal

Author: Dirk Van Compernolle   
Created: 29/03/2021

---------------------------------------------------------
Dependencies: 
- IPython > 6.0       (works with lower version, but not fully)
- ipywidget
- matplotlib
- numpy 
- scipy 
- librosa

In [1]:
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive, interactive_output
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from IPython.display import display, clear_output, Audio, HTML
import time
import librosa
import pyspch.spectrogram as specg
import pyspch.display as spch_disp

def my_sine(x, w, amp, phi):
    return amp*np.sin(w * (x-phi))
#
def synth(sigtype='sin', freq=200.0, amp=1.0, phi=0.0, sample_rate=8000, Tmax=0.25):
    t = np.linspace(0.0, Tmax, int(Tmax*sample_rate), endpoint=False)
    tt = 2.*np.pi*(freq*t)
    if sigtype == 'sin':
        y = np.sin(tt)
    elif sigtype == 'square':
        y = signal.square(tt + phi)
    elif sigtype == 'sawtooth':
        y = signal.sawtooth(tt + phi )
    elif sigtype == 'chirp 1:20':
        y = signal.chirp(t,freq,Tmax,20.*freq,method='linear')
    elif sigtype == 'chirp 20:1':
        y = signal.chirp(t,20*freq,Tmax,freq,method='linear')
    elif sigtype == 'gausspulse':
        y = signal.gausspulse(t-Tmax/2.,fc=freq,bw=.4)
    elif sigtype == 'modulated white noise':
        y = np.random.randn(len(t))*np.sin(tt)
    elif sigtype == 'Dual Tone':   #DTMF tone ratios is approximately 21/19 ~ 1.1054 (697,770,852,941)*(1209,1336,1477)
                              # row/col ratio is 
        tt1 = 2.*np.pi*(1.735*freq*t)
        y = 0.5*np.sin(tt) + 0.5*np.sin(tt1)   
    else:
        print( 'signal: Unrecognized signal type')
    return amp*y, t


# a default boxed layout
def box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )

# The following code will increase the default width of your Jupyter notebook cells
# Supposed to work well 
display(HTML(data="""
<style>
    div#notebook-container    { width: 99%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [2]:
class Harmonic_Signals(widgets.HBox):

    def __init__(self,Tmax=.3,sample_rate=16000,freq_range=[100.,500.]):
        super().__init__()
        
        if sample_rate < 4000.:
            print("WARNING: sampling rate should not be lower than 4000.0 Hz and has been reset.")
            sample_rate = 4000.0
        self.sample_rate = sample_rate
        self.nfft = 512
        self.Tmax = Tmax        
        display_types = ['spectrum','spectrogram']
        signal_types = [ 'sin', 'square', 'sawtooth', 'chirp 1:20','chirp 20:1', 'gausspulse','modulated white noise','Dual Tone']
        
        self.disptype = 'spectrum'
        self.sigtype = 'sin'
        self.freq = 110.
        self.amp = 1.
        self.phi = 0.
        self.autoplay = False

        # create the widgets
        self.wg_disptype = widgets.Dropdown(options=display_types,value=self.disptype,description="Display")
        self.wg_sigtype = widgets.Dropdown(options=signal_types,value=self.sigtype,description="Signal")
        self.wg_amp = widgets.FloatLogSlider(value=self.amp,step=0.2,min=-2.,max=0.0,description='Amplitude',continous_update=False)
        freq_step = 10. if (freq_range[1]-freq_range[0]) > 50 else 1.  
        self.wg_freq = widgets.FloatSlider(value=self.freq,step=freq_step,min=freq_range[0],max=freq_range[1],description='Frequency',continous_update=False)
        self.wg_phi = widgets.FloatSlider(value=self.phi,step=0.1,min=0.0,max=2.*np.pi,description='Phase',continous_update=False)
        self.wg_autoplay = widgets.Checkbox(value=self.autoplay,description='Autoplay Audio',indent=False,button_style='warning')
        self.wg_autoplay.layout.width='50%'
        self.wg_clear_log = widgets.Button(description='Clear the log')
        self.wg_clear_log.layout.width='50%'
        
        # link to the widget observers
        self.wg_disptype.observe(self.disptype_observe,'value')
        self.wg_sigtype.observe(self.sigtype_observe,'value')
        self.wg_amp.observe(self.amp_observe,'value')    
        self.wg_freq.observe(self.freq_observe,'value')
        self.wg_phi.observe(self.phi_observe,'value')
        self.wg_autoplay.observe(self.autoplay_clicked, 'value')
        self.wg_clear_log.on_click(self.clear_log_clicked)

        # setup the outputs 
        self.audio = widgets.Output()
        self.logscr = widgets.Output()
        self.out = widgets.Output(layout=box_layout())
        self.UI = widgets.VBox( [self.wg_disptype,self.wg_sigtype, self.wg_amp, self.wg_freq, self.wg_phi,self.wg_autoplay],
                                 layout=box_layout())
        self.right_screen = widgets.VBox([self.UI, self.audio, self.wg_clear_log,  self.logscr],layout=box_layout())
                                          
        # initialize the plots
        #self.fig,self.ax = plt.subplots(2,1,constrained_layout=True, figsize=(9, 4))
        self.fig = spch_disp.make_subplots(row_heights=[1.,1.], figsize=(6, 4),dpi=100)        
        self.update()
        plt.close()          # avoids output of dummy figure on startup
        # initialize the displayed function

        # attach children to the VBox class
        self.children = [ self.out, self.right_screen ] 

    def update(self):
        y,x = synth(sigtype=self.sigtype,freq=self.freq,phi=self.phi,amp=self.amp,sample_rate=self.sample_rate,Tmax=self.Tmax)

        ax = self.fig.axes
       
        if self.disptype == 'spectrum':
            ax[0].cla()
            ax[1].cla()
            spch_disp.add_line_plot(ax[0],y,x,yrange=[-1.,1.],xlabel='Time(sec)',title='Waveform')
            freq_ax,spec = signal.periodogram(y,fs=self.sample_rate,scaling='spectrum')
            ax[1].plot(freq_ax,np.sqrt(2*spec))
            ax[1].set_title('Spectrum')
            ax[1].set_xlabel('Frequency (Hz)')
            ax[1].set_ylim([0, 1])
            ax[1].grid(True)
            ax[1].set_xlim([0,self.sample_rate/2.])
        elif self.disptype == 'spectrogram':
            spg = specg.spectrogram(y,sample_rate=self.sample_rate,preemp=0.)
            yax = np.arange(self.nfft/2 + 1)*(self.sample_rate/self.nfft) 
            spch_disp.plot_spg(spg,fig=self.fig,wav=y,sample_rate=self.sample_rate,yax=yax)
            ax[0].set_title('Waveform')
            ax[0].set_xlabel('')
            ax[1].set_title('Spectrogram')
            ax[1].set_xlabel('Time(sec)')
            #ax[1].set_ylabel('Hz')

            
        # here come the things that go to dedicated output widgets
        with self.out:
            clear_output(wait=True)
            display(self.fig)
        with self.audio:
            clear_output(wait=True)
            try:
                display(Audio(data=y,rate=self.sample_rate,normalize=False, autoplay=self.autoplay))
            except: 
                try:
                    with self.logscr:
                        #clear_output(wait=True)
                        print("Warning: playing normalized sound")
                    display(Audio(data=y,rate=self.sample_rate, autoplay=self.autoplay))
                except: pass
     
    def disptype_observe(self,change):
        self.disptype = change.new
        self.update()
    
    def sigtype_observe(self,change):
        self.sigtype = change.new
        self.update()
    
    def amp_observe(self,change):
        self.amp = change.new
        self.update()
        
    def freq_observe(self,change):
        self.freq = change.new
        self.update()
        
    def phi_observe(self,change):
        self.phi = change.new
        self.update()
        
    def autoplay_clicked(self,obj):
        self.autoplay = not self.autoplay
        with self.logscr: print('Toggling autoplay',self.autoplay) 

    def clear_log_clicked(self,b):
        with self.logscr: clear_output()                   



## DEMO 1: Waveform and Spectrum of Harmonic Signals


In this demo periodic signals are generated that can be characterized as a harmonic complex of sine waves.

$y= A \sum_k \alpha_k  sin(2  \pi k f_0 t+\Phi)$

In addition signals can be generated with either time-varying amplitude (e.g. Gaussian pulse) or time-varying frequency (e.g. chirps). 

You can control:
- The fundamental frequency $f_0$  (named "frequency" in the control panel)  
- The overall amplitude $A$
- The relative amplitude of the harmonics $\alpha_k$ by choosing from a set of predefined harmonic functions
- The global phase $\Phi$  (you cannot control relative phase differences per harmonic component)

Signal Types to choose from:
- stationary and periodic
    + sine wave
    + square 
    + sawtooth
- non stationary signals:
    + chirp 1:10 : ramping up the frequency by a factor of 20
    + chirp 20:1 : ramping down the frequency by a factor of 20
    + gausspulse

    
Displays to choose from:
- spectrum: most appropiate for stationary signals
- spectrogram: essential for non-stationary signals
    
CAVEAT:   
+ The Autoplay function does not always work properly on Ipython < 6.0 [Google Colab] which has autonormalization by default in the display.Audio() function

In [4]:
Harmonic_Signals(Tmax=0.5,sample_rate=8000,freq_range=[100,300.]) 

Harmonic_Signals(children=(Output(layout=Layout(border='solid 1px black', margin='0px 10px 10px 0px', padding=…